# XBRL2 - Querying XBRL Facts with Enhanced API

This notebook demonstrates how to use the enhanced Facts module in the XBRL2 API to query and analyze XBRL facts in various ways. 

The Facts module lets you directly access and filter individual XBRL facts using a flexible query interface. Key features include:

1. **Filter by statement type** - Get facts from specific financial statements
2. **Filter by period views** - Use predefined period selections for easier time-based analysis
3. **Smart text search** - Search across concept names, labels, and elements 
4. **Safe numeric filtering** - Properly handle None values in numeric comparisons
5. **Period and dimension filtering** - Analyze facts across time periods and dimensions
6. **Pandas integration** - Easy conversion to DataFrames for further analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/XBRL2-FactQueries.ipynb)

In [ ]:
!pip install edgartools

In [21]:
from edgar import *
from edgar.xbrl2 import XBRL, FactQuery, FactsView
import pandas as pd

set_identity("PeriodViews@notebook.com")

## Load a Filing with XBRL Data

In [22]:
company = Company('AAPL')
filing = company.latest("10-K")  # Get the latest 10-K filing

In [23]:
# Parse the XBRL data
xbrl = XBRL.from_filing(filing)

## Basic Facts Access

The `facts_view` property provides access to all facts in the XBRL document.

In [24]:
# Get the facts view
facts = xbrl.facts_view

# Get a summary of the facts
summary = facts.summarize()
print(f"Total facts: {summary['total_facts']}")
print(f"\nFacts by statement type:")
for stmt, count in summary['by_statement'].items():
    print(f"  {stmt}: {count}")

print(f"\nFacts by period type:")
for period_type, count in summary['by_period_type'].items():
    print(f"  {period_type}: {count}")

print(f"\nUnique dimensions: {len(summary['dimensions'])}")
if summary['dimensions']:
    for dim in summary['dimensions'][:5]:  # Show first 5
        print(f"  {dim}")
    if len(summary['dimensions']) > 5:
        print(f"  ...and {len(summary['dimensions']) - 5} more")

Total facts: 1042

Facts by statement type:
  unknown: 495
  IncomeStatement: 272
  BalanceSheet: 162
  StatementOfEquity: 28
  CashFlowStatement: 85

Facts by period type:
  duration: 550
  instant: 492

Unique dimensions: 25
  ecd_IndividualAxis
  srt_ConsolidationItemsAxis
  srt_MajorCustomersAxis
  srt_ProductOrServiceAxis
  srt_RangeAxis
  ...and 20 more


## Querying Facts

The `FactQuery` provides a fluent interface for filtering facts by various criteria.

In [25]:
# Find revenue-related facts
revenue_df = facts.query().by_concept("Revenue").to_dataframe()
revenue_df[['concept',  'numeric_value', 'period_end']].head(10)

,concept,numeric_value,period_end
0,us-gaap:RevenueRemainingPerformanceObligationE...,NaN,NaN
1,us-gaap:RevenueRemainingPerformanceObligationE...,NaN,NaN
2,us-gaap:RevenueRemainingPerformanceObligationE...,NaN,NaN
3,us-gaap:RevenueRemainingPerformanceObligationE...,NaN,NaN
4,us-gaap:RevenueFromContractWithCustomerExcludi...,2.948660e+11,2024-09-28
5,us-gaap:RevenueFromContractWithCustomerExcludi...,2.980850e+11,2023-09-30
6,us-gaap:RevenueFromContractWithCustomerExcludi...,3.161990e+11,2022-09-24
7,us-gaap:RevenueFromContractWithCustomerExcludi...,9.616900e+10,2024-09-28
8,us-gaap:RevenueFromContractWithCustomerExcludi...,8.520000e+10,2023-09-30
9,us-gaap:RevenueFromContractWithCustomerExcludi...,7.812900e+10,2022-09-24


## Query By Label

In [26]:
# Improved text search function
# This feature allows searching across all text fields (concept, label, element_name, etc.)
# and handles NULL values appropriately

# Search for "Revenue" in any text field
revenue_results = facts.search_facts("Revenue")
print(f"Found {len(revenue_results)} facts containing 'Revenue' in any text field")

# Display the first few results
revenue_results[['concept', 'label', 'numeric_value', 'period_end']].head(5)

Found 60 facts containing 'Revenue' in any text field


,concept,label,numeric_value,period_end
0,us-gaap:RevenueRemainingPerformanceObligationE...,"Revenue, Remaining Performance Obligation, Exp...",NaN,NaN
1,us-gaap:RevenueRemainingPerformanceObligationE...,"Revenue, Remaining Performance Obligation, Exp...",NaN,NaN
2,us-gaap:RevenueRemainingPerformanceObligationE...,"Revenue, Remaining Performance Obligation, Exp...",NaN,NaN
3,us-gaap:RevenueRemainingPerformanceObligationE...,"Revenue, Remaining Performance Obligation, Exp...",NaN,NaN
4,us-gaap:RevenueFromContractWithCustomerExcludi...,"Revenue from Contract with Customer, Excluding...",2.948660e+11,2024-09-28


## Query by text

In [27]:
(facts.query().by_text("Tax")
 .to_dataframe().head(10))

,element_id,context_ref,value,unit_ref,decimals,numeric_value,footnotes,fact_key,concept,period_type,...,statement_role,period_instant,dim_us-gaap_StatementEquityComponentsAxis,dim_us-gaap_FairValueByFairValueHierarchyLevelAxis,dim_us-gaap_FinancialInstrumentAxis,dim_us-gaap_LossContingenciesByNatureOfContingencyAxis,dim_srt_RangeAxis,dim_us-gaap_AwardTypeAxis,dim_us-gaap_StatementBusinessSegmentsAxis,dim_srt_StatementGeographicalAxis
0,dei:EntityTaxIdentificationNumber,c-1,94-2404110,None,None,NaN,[],dei:EntityTaxIdentificationNumber_c-1,dei:EntityTaxIdentificationNumber,duration,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,us-gaap:RevenueFromContractWithCustomerExcludi...,c-13,294866000000,usd,-6,2.948660e+11,[],us-gaap:RevenueFromContractWithCustomerExcludi...,us-gaap:RevenueFromContractWithCustomerExcludi...,duration,...,http://www.apple.com/role/CONSOLIDATEDSTATEMEN...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,us-gaap:RevenueFromContractWithCustomerExcludi...,c-14,298085000000,usd,-6,2.980850e+11,[],us-gaap:RevenueFromContractWithCustomerExcludi...,us-gaap:RevenueFromContractWithCustomerExcludi...,duration,...,http://www.apple.com/role/CONSOLIDATEDSTATEMEN...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,us-gaap:RevenueFromContractWithCustomerExcludi...,c-15,316199000000,usd,-6,3.161990e+11,[],us-gaap:RevenueFromContractWithCustomerExcludi...,us-gaap:RevenueFromContractWithCustomerExcludi...,duration,...,http://www.apple.com/role/CONSOLIDATEDSTATEMEN...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,us-gaap:RevenueFromContractWithCustomerExcludi...,c-16,96169000000,usd,-6,9.616900e+10,[],us-gaap:RevenueFromContractWithCustomerExcludi...,us-gaap:RevenueFromContractWithCustomerExcludi...,duration,...,http://www.apple.com/role/CONSOLIDATEDSTATEMEN...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,us-gaap:RevenueFromContractWithCustomerExcludi...,c-17,85200000000,usd,-6,8.520000e+10,[],us-gaap:RevenueFromContractWithCustomerExcludi...,us-gaap:RevenueFromContractWithCustomerExcludi...,duration,...,http://www.apple.com/role/CONSOLIDATEDSTATEMEN...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,us-gaap:RevenueFromContractWithCustomerExcludi...,c-18,78129000000,usd,-6,7.812900e+10,[],us-gaap:RevenueFromContractWithCustomerExcludi...,us-gaap:RevenueFromContractWithCustomerExcludi...,duration,...,http://www.apple.com/role/CONSOLIDATEDSTATEMEN...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,us-gaap:RevenueFromContractWithCustomerExcludi...,c-1,391035000000,usd,-6,3.910350e+11,[],us-gaap:RevenueFromContractWithCustomerExcludi...,us-gaap:RevenueFromContractWithCustomerExcludi...,duration,...,http://www.apple.com/role/CONSOLIDATEDSTATEMEN...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,us-gaap:RevenueFromContractWithCustomerExcludi...,c-19,383285000000,usd,-6,3.832850e+11,[],us-gaap:RevenueFromContractWithCustomerExcludi...,us-gaap:RevenueFromContractWithCustomerExcludi...,duration,...,http://www.apple.com/role/CONSOLIDATEDSTATEMEN...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,us-gaap:RevenueFromContractWithCustomerExcludi...,c-20,394328000000,usd,-6,3.943280e+11,[],us-gaap:RevenueFromContractWithCustomerExcludi...,us-gaap:RevenueFromContractWithCustomerExcludi...,duration,...,http://www.apple.com/role/CONSOLIDATEDSTATEMEN...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Query by instant

In [28]:
facts.query().by_period_type("instant").to_dataframe().head(10)

,element_id,context_ref,value,unit_ref,decimals,numeric_value,footnotes,fact_key,concept,period_type,...,dim_us-gaap_PropertyPlantAndEquipmentByTypeAxis,dim_srt_RangeAxis,dim_us-gaap_UnrecordedUnconditionalPurchaseObligationByCategoryOfItemPurchasedAxis,dim_us-gaap_ShortTermDebtTypeAxis,dim_us-gaap_DebtInstrumentAxis,dim_us-gaap_LongtermDebtTypeAxis,dim_us-gaap_PlanNameAxis,dim_us-gaap_AwardTypeAxis,dim_srt_StatementGeographicalAxis,dim_ecd_IndividualAxis
0,us-gaap:RevenueRemainingPerformanceObligationE...,c-58,P1Y,None,None,NaN,[],us-gaap:RevenueRemainingPerformanceObligationE...,us-gaap:RevenueRemainingPerformanceObligationE...,instant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,us-gaap:RevenueRemainingPerformanceObligationE...,c-59,P1Y,None,None,NaN,[],us-gaap:RevenueRemainingPerformanceObligationE...,us-gaap:RevenueRemainingPerformanceObligationE...,instant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,us-gaap:RevenueRemainingPerformanceObligationE...,c-60,P1Y,None,None,NaN,[],us-gaap:RevenueRemainingPerformanceObligationE...,us-gaap:RevenueRemainingPerformanceObligationE...,instant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,us-gaap:RevenueRemainingPerformanceObligationE...,c-61,P1Y,None,None,NaN,[],us-gaap:RevenueRemainingPerformanceObligationE...,us-gaap:RevenueRemainingPerformanceObligationE...,instant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,us-gaap:HedgedAssetStatementOfFinancialPositio...,c-21,http://fasb.org/us-gaap/2024#MarketableSecurit...,None,None,NaN,[],us-gaap:HedgedAssetStatementOfFinancialPositio...,us-gaap:HedgedAssetStatementOfFinancialPositio...,instant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,us-gaap:HedgedAssetStatementOfFinancialPositio...,c-22,http://fasb.org/us-gaap/2024#MarketableSecurit...,None,None,NaN,[],us-gaap:HedgedAssetStatementOfFinancialPositio...,us-gaap:HedgedAssetStatementOfFinancialPositio...,instant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,us-gaap:HedgedLiabilityStatementOfFinancialPos...,c-21,http://fasb.org/us-gaap/2024#LongTermDebtCurre...,None,None,NaN,[],us-gaap:HedgedLiabilityStatementOfFinancialPos...,us-gaap:HedgedLiabilityStatementOfFinancialPos...,instant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,us-gaap:HedgedLiabilityStatementOfFinancialPos...,c-22,http://fasb.org/us-gaap/2024#LongTermDebtCurre...,None,None,NaN,[],us-gaap:HedgedLiabilityStatementOfFinancialPos...,us-gaap:HedgedLiabilityStatementOfFinancialPos...,instant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,us-gaap:OperatingLeaseRightOfUseAssetStatement...,c-21,http://fasb.org/us-gaap/2024#OtherAssetsNoncur...,None,None,NaN,[],us-gaap:OperatingLeaseRightOfUseAssetStatement...,us-gaap:OperatingLeaseRightOfUseAssetStatement...,instant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,us-gaap:OperatingLeaseRightOfUseAssetStatement...,c-22,http://fasb.org/us-gaap/2024#OtherAssetsNoncur...,None,None,NaN,[],us-gaap:OperatingLeaseRightOfUseAssetStatement...,us-gaap:OperatingLeaseRightOfUseAssetStatement...,instant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Query by duration

In [29]:
facts.query().by_period_type("duration").to_dataframe().head(10)

,element_id,context_ref,value,unit_ref,decimals,numeric_value,footnotes,fact_key,concept,period_type,...,dim_us-gaap_ConcentrationRiskByTypeAxis,dim_us-gaap_LossContingenciesByNatureOfContingencyAxis,dim_us-gaap_ShortTermDebtTypeAxis,dim_us-gaap_DebtInstrumentAxis,dim_us-gaap_LongtermDebtTypeAxis,dim_us-gaap_AwardTypeAxis,dim_us-gaap_PlanNameAxis,dim_us-gaap_StatementBusinessSegmentsAxis,dim_srt_ConsolidationItemsAxis,dim_srt_StatementGeographicalAxis
0,dei:AmendmentFlag,c-1,false,None,None,NaN,[],dei:AmendmentFlag_c-1,dei:AmendmentFlag,duration,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,dei:DocumentFiscalYearFocus,c-1,2024,None,None,2024.0,[],dei:DocumentFiscalYearFocus_c-1,dei:DocumentFiscalYearFocus,duration,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,dei:DocumentFiscalPeriodFocus,c-1,FY,None,None,NaN,[],dei:DocumentFiscalPeriodFocus_c-1,dei:DocumentFiscalPeriodFocus,duration,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,dei:EntityCentralIndexKey,c-1,0000320193,None,None,320193.0,[],dei:EntityCentralIndexKey_c-1,dei:EntityCentralIndexKey,duration,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TrdArrDuration,c-189,P856D,None,None,NaN,[],TrdArrDuration_c-189,TrdArrDuration,duration,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,TrdArrDuration,c-192,P473D,None,None,NaN,[],TrdArrDuration_c-192,TrdArrDuration,duration,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,dei:DocumentType,c-1,10-K,None,None,NaN,[],dei:DocumentType_c-1,dei:DocumentType,duration,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,dei:DocumentAnnualReport,c-1,true,None,None,NaN,[],dei:DocumentAnnualReport_c-1,dei:DocumentAnnualReport,duration,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,dei:DocumentPeriodEndDate,c-1,2024-09-28,None,None,NaN,[],dei:DocumentPeriodEndDate_c-1,dei:DocumentPeriodEndDate,duration,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,dei:CurrentFiscalYearEndDate,c-1,--09-28,None,None,NaN,[],dei:CurrentFiscalYearEndDate_c-1,dei:CurrentFiscalYearEndDate,duration,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Query by statement type

In [30]:
# Safe numeric value filtering with improved by_value method
# This function now safely handles None values and properly compares numeric values

# Find values over 10 billion (safe handling of None values)
large_values = facts.query() \
    .by_statement_type('IncomeStatement') \
    .by_value(lambda v: v > 10_000_000_000) \
    .sort_by('numeric_value', ascending=False) \
    .to_dataframe()

print(f"Found {len(large_values)} values over 10 billion")
large_values[['concept', 'label', 'numeric_value', 'period_key']].head(5)

# Filter using a range
mid_values = facts.query() \
    .by_statement_type('IncomeStatement') \
    .by_value((1_000_000_000, 10_000_000_000)) \
    .sort_by('numeric_value', ascending=False) \
    .to_dataframe()

print(f"\nFound {len(mid_values)} values between 1-10 billion")
mid_values[['concept', 'label', 'numeric_value', 'period_key']].head(5)

Found 141 values over 10 billion

Found 70 values between 1-10 billion


,concept,label,numeric_value,period_key
0,us-gaap:UnrecognizedTaxBenefitsThatWouldImpact...,Unrecognized Tax Benefits that Would Impact Ef...,9.500000e+09,instant_2023-09-30
1,us-gaap:CurrentFederalTaxExpenseBenefit,Current Federal Tax Expense (Benefit),9.445000e+09,duration_2022-09-25_2023-09-30
2,us-gaap:ForeignIncomeTaxExpenseBenefitContinui...,"Foreign Income Tax Expense (Benefit), Continui...",9.419000e+09,duration_2022-09-25_2023-09-30
3,us-gaap:AllocatedShareBasedCompensationExpense,"Share-Based Payment Arrangement, Expense",9.038000e+09,duration_2021-09-26_2022-09-24
4,us-gaap:CurrentForeignTaxExpenseBenefit,Current Foreign Tax Expense (Benefit),8.996000e+09,duration_2021-09-26_2022-09-24


In [31]:
# Find facts from the income statement with values greater than 10 billion
large_values = (facts.query()
    .by_statement_type('IncomeStatement')
    .by_value(lambda v: v > 10_000_000_000)
    .sort_by('numeric_value', ascending=False)
    .to_dataframe()
)

large_values[['concept', 'value', 'period_start', 'period_end']].head(10)

,concept,value,period_start,period_end
0,us-gaap:RevenueFromContractWithCustomerExcludi...,394328000000,2021-09-26,2022-09-24
1,us-gaap:RevenueFromContractWithCustomerExcludi...,391035000000,2023-10-01,2024-09-28
2,us-gaap:RevenueFromContractWithCustomerExcludi...,383285000000,2022-09-25,2023-09-30
3,us-gaap:RevenueFromContractWithCustomerExcludi...,316199000000,2021-09-26,2022-09-24
4,us-gaap:RevenueFromContractWithCustomerExcludi...,298085000000,2022-09-25,2023-09-30
5,us-gaap:RevenueFromContractWithCustomerExcludi...,294866000000,2023-10-01,2024-09-28
6,us-gaap:CostOfGoodsAndServicesSold,223546000000,2021-09-26,2022-09-24
7,us-gaap:CostOfGoodsAndServicesSold,214137000000,2022-09-25,2023-09-30
8,us-gaap:CostOfGoodsAndServicesSold,210352000000,2023-10-01,2024-09-28
9,us-gaap:RevenueFromContractWithCustomerExcludi...,205489000000,2021-09-26,2022-09-24


## Query by period views

In [32]:
facts.get_facts_by_period_view("IncomeStatement",
                               "Three Recent Quarters").head(10)

""


In [33]:
# Improved period filtering with by_period_key and by_period_keys methods

# Let's get the available reporting periods
all_periods = xbrl.reporting_periods
print(f"Filing has {len(all_periods)} reporting periods")

# Get the most recent instant period (for balance sheet)
latest_period = next((p for p in all_periods if p['type'] == 'instant'), 
                     None)
if latest_period:
    print(f"Latest instant period: {latest_period['date']}")
    
    # Get assets from the balance sheet for this period
    assets = facts.query() \
        .by_statement_type('BalanceSheet') \
        .by_period_key(latest_period['key']) \
        .by_label("Asset", exact=False) \
        .to_dataframe()
    
    print(f"\nFound {len(assets)} asset-related facts for period {latest_period['key']}")
    display(assets[['concept', 'label', 'numeric_value']].head(5))
    
# Let's get the last 2 periods for the income statement
duration_periods = [p for p in all_periods if p['type'] == 'duration']
if len(duration_periods) >= 2:
    period_keys = [p['key'] for p in duration_periods[:2]]
    print(f"\nComparing periods: {period_keys}")
    
    # Get income data for these two periods
    income_comparison = facts.query() \
        .by_statement_type('IncomeStatement') \
        .by_period_keys(period_keys) \
        .by_label("Revenue", exact=False) \
        .to_dataframe()
    
    print(f"Found {len(income_comparison)} revenue facts across both periods")
    
    # Create a pivot table to show the comparison
    pivot = income_comparison.pivot_table(
        values='numeric_value',
        index=['concept', 'label'],
        columns='period_key',
        aggfunc='first'
    )
    
    display(pivot.head(5))

Filing has 13 reporting periods
Latest instant period: 2024-10-18

Found 0 asset-related facts for period instant_2024-10-18


KeyError: "None of [Index(['concept', 'label', 'numeric_value'], dtype='object')] are in the [columns]"

In [34]:
xbrl.get_period_views("IncomeStatement")

[{'name': 'Three-Year Comparison',
  'description': 'Compares three fiscal years',
  'period_keys': ['duration_2023-10-01_2024-09-28',
   'duration_2022-09-25_2023-09-30',
   'duration_2021-09-26_2022-09-24']},
 {'name': 'Annual Comparison',
  'description': 'Compares recent fiscal years',
  'period_keys': ['duration_2023-10-01_2024-09-28',
   'duration_2022-09-25_2023-09-30']}]

## Working with Specific Facts

In [35]:
# Get all facts from the balance sheet
balance_sheet_facts = facts.get_statement_facts('BalanceSheet')
print(f"Balance sheet has {len(balance_sheet_facts)} facts")

# Show some key balance sheet facts
balance_sheet_facts[balance_sheet_facts['label'].str.contains('Total Assets|Total Liabilities|Stockholders')][['concept', 'label', 'numeric_value']].head(10)

Balance sheet has 162 facts


,concept,label,numeric_value


## Time Series Analysis

## Period Views

XBRL data typically contains multiple reporting periods. The Facts module allows you to query facts using predefined period views.

In [36]:
# Get available period views for the income statement
income_views = facts.get_available_period_views('IncomeStatement')
print("Available period views for Income Statement:")
for view in income_views:
    print(f"- {view['name']}: {view['description']}")
    print(f"  Periods: {view['period_keys']}")
    print("")

# Also check balance sheet views
balance_sheet_views = facts.get_available_period_views('BalanceSheet')
print("\nAvailable period views for Balance Sheet:")
for view in balance_sheet_views:
    print(f"- {view['name']}: {view['description']}")
    print(f"  Periods: {view['period_keys']}")
    print("")

Available period views for Income Statement:
- Three-Year Comparison: Compares three fiscal years
  Periods: ['duration_2023-10-01_2024-09-28', 'duration_2022-09-25_2023-09-30', 'duration_2021-09-26_2022-09-24']

- Annual Comparison: Compares recent fiscal years
  Periods: ['duration_2023-10-01_2024-09-28', 'duration_2022-09-25_2023-09-30']


Available period views for Balance Sheet:
- Three-Year Comparison: Shows the most recent three periods
  Periods: ['instant_2024-10-18', 'instant_2024-09-28', 'instant_2024-03-29']

- Current vs. Previous Periods: Shows the current period and two previous periods
  Periods: ['instant_2024-10-18', 'instant_2024-09-28', 'instant_2024-03-29']

- Three-Year Annual Comparison: Shows three fiscal years for comparison
  Periods: ['instant_2024-10-18', 'instant_2024-09-28', 'instant_2023-09-30']

- Annual Comparison: Shows two fiscal years for comparison
  Periods: ['instant_2024-10-18', 'instant_2024-09-28']



# Query facts using a specific period view
if income_views:  # Make sure there are available views
    # Let's use Annual Comparison if available, otherwise use the first view
    annual_view = next((view for view in income_views if 'Annual' in view['name']), income_views[0])
    view_name = annual_view['name']
    print(f"Getting facts for period view: {view_name}")
    
    # Get facts filtered by this period view
    view_facts = facts.get_facts_by_period_view('IncomeStatement', view_name)
    
    # Get key metrics like revenue and net income
    key_metrics = view_facts[view_facts['label'].str.contains('Revenue|Net Income|Operating Income', 
                                                             case=False, na=False)]
    
    # Show the results
    print(f"Found {len(key_metrics)} key metrics across {len(annual_view['period_keys'])} periods")
    display(key_metrics[['concept', 'label', 'numeric_value', 'period_key']].head(10))
    
    # Create a pivot table to better visualize the data across periods
    pivot = key_metrics.pivot_table(
        values='numeric_value',
        index=['concept', 'label'],
        columns='period_key',
        aggfunc='first'
    )
    
    # Display the pivoted data
    print("\nPivot table of key metrics across periods:")
    display(pivot)

## Time Series Analysis

In [37]:
# If we have a useful dimension, create a pivoted view
if dimensions:
    # Use the first dimension as an example
    dim_name = list(dimensions.keys())[0]
    pivot_df = facts.pivot_by_dimension(dim_name)
    if not pivot_df.empty:
        display(pivot_df.head())

NameError: name 'dimensions' is not defined

# Compare quarterly periods if available
quarterly_view = next((view for view in income_views if 'Quarter' in view['name']), None)
if quarterly_view:
    print(f"Analyzing quarterly comparison: {quarterly_view['name']}")
    
    # Get facts for quarterly comparison
    quarterly_facts = facts.get_facts_by_period_view('IncomeStatement', quarterly_view['name'])
    
    # Focus on revenue
    revenue_facts = quarterly_facts[quarterly_facts['label'].str.contains('Revenue', case=False, na=False)]
    
    # Show quarterly revenue
    display(revenue_facts[['concept', 'label', 'numeric_value', 'period_key']].head(10))
    
    # Pivot by quarter
    revenue_pivot = revenue_facts.pivot_table(
        values='numeric_value',
        index=['concept', 'label'],
        columns='period_key',
        aggfunc='first'
    )
    
    # Display the quarterly comparison
    print("\nQuarterly Revenue Comparison:")
    display(revenue_pivot)
    
    # Plot if we have at least 2 periods
    if len(quarterly_view['period_keys']) >= 2 and not revenue_pivot.empty:
        # Convert the pivot table to be more plot-friendly
        plot_df = revenue_pivot.reset_index()
        
        # Only plot the revenue row (not sub-components)
        main_revenue = plot_df[plot_df['label'].str.contains('^Revenue$|^Total Revenue$', case=False, regex=True)]
        
        if not main_revenue.empty:
            # Melt the dataframe to get it in the right format for plotting
            period_columns = [col for col in main_revenue.columns if col not in ['concept', 'label']]
            plot_ready = main_revenue.melt(
                id_vars=['concept', 'label'],
                value_vars=period_columns,
                var_name='Period',
                value_name='Revenue'
            )
            
            # Plot
            ax = plot_ready.plot(
                x='Period', 
                y='Revenue', 
                kind='bar', 
                figsize=(12, 6), 
                title=f"Revenue by Quarter"
            )
            ax.set_ylabel('Revenue ($)')
            ax.set_xlabel('Period')
            ax.grid(axis='y')

In [ ]:
# Get income statement items across periods
income_pivot = facts.pivot_by_period(statement_type='IncomeStatement')
if not income_pivot.empty:
    # Filter to just a few key metrics
    key_metrics = income_pivot[income_pivot['label'].str.contains('Revenue|Income|Earnings', case=False, na=False)]
    display(key_metrics.head(10))

## Complex Queries

Combining multiple filters allows for powerful and specific queries.

In [38]:
# Find facts that are:
# 1. Related to the balance sheet
# 2. Have "cash" in their label
# 3. Are for the most recent period
# 4. Have a value greater than 1 billion

# First, find the most recent period in the balance sheet
bs_periods = [p for p in summary['periods'] if 'instant' in p]
if bs_periods:
    latest_period = sorted(bs_periods)[-1]  # Get the last period when sorted
    
    complex_query = facts.query()\
        .by_statement_type('BalanceSheet')\
        .by_label('cash', exact=False)\
        .by_custom(lambda f: 'period_key' in f and f['period_key'] == latest_period)\
        .by_value(lambda v: v > 1_000_000_000)\
        .sort_by('numeric_value', ascending=False)
    
    result = complex_query.to_dataframe()
    display(result[['concept', 'label', 'numeric_value', 'period_key']].head())

KeyError: "None of [Index(['concept', 'label', 'numeric_value', 'period_key'], dtype='object')] are in the [columns]"

## Combining with Traditional Statement Access

The facts module complements the existing statements functionality, allowing you to analyze the same data in different ways.

In [39]:
# Get the balance sheet using the statements API
balance_sheet = xbrl.statements.balance_sheet()
print(balance_sheet)

                           CONSOLIDATEDBALANCESHEETS (Standardized)                            
                  Fiscal Year Ended (In millions, except shares in thousands)                  
                                                                                               
  Line Item                                        Sep 28, 2024   Sep 30, 2023   Sep 24, 2022  
 ───────────────────────────────────────────────────────────────────────────────────────────── 
    ASSETS:                                                                                    
      Current assets:                                                                          
        Cash and Cash Equivalents                       $29,943        $29,965                 
        Marketable securities                           $35,228        $31,590                 
        Accounts Receivable                             $33,410        $29,508                 
        Vendor non-trade receivables    

In [40]:
# Compare with facts-based approach for the same data
cash_assets = facts.query()\
    .by_statement_type('BalanceSheet')\
    .by_label(r'[Cc]ash|[Ee]quivalent')\
    .to_dataframe()

display(cash_assets[['concept', 'label', 'numeric_value', 'period_instant']].head())

,concept,label,numeric_value,period_instant
0,us-gaap:CashAndCashEquivalentsAtCarryingValue,"Cash and Cash Equivalents, at Carrying Value",2.994300e+10,2024-09-28
1,us-gaap:CashAndCashEquivalentsAtCarryingValue,"Cash and Cash Equivalents, at Carrying Value",2.996500e+10,2023-09-30
2,us-gaap:CashAndCashEquivalentsAtCarryingValue,"Cash and Cash Equivalents, at Carrying Value",2.719900e+10,2024-09-28
3,us-gaap:CashAndCashEquivalentsAtCarryingValue,"Cash and Cash Equivalents, at Carrying Value",7.780000e+08,2024-09-28
4,us-gaap:CashAndCashEquivalentsAtCarryingValue,"Cash and Cash Equivalents, at Carrying Value",0.000000e+00,2024-09-28


## Conclusion

The enhanced Facts module provides a flexible and powerful way to query and analyze XBRL data, complementing the statement-oriented approach of the core XBRL2 API. You can use it to:

1. Search for specific concepts, labels, or values with robust handling of null values
2. Analyze facts across multiple dimensions and time periods
3. Use smart text search across multiple fields with a single query
4. Filter by predefined period views or custom period selections
5. Combine multiple filters for precise data selection
6. Generate pandas DataFrames for further analysis and visualization

Recent enhancements include:
- Improved text search across multiple fields with `search_facts()` method
- Safer numeric value filtering with proper null-value handling
- New period filtering methods including `by_period_key()` and `by_period_keys()`
- Enhanced period views with facts count statistics
- Better handling of namespaced elements with colon/underscore conversion

This query-oriented approach is particularly useful for ad-hoc analysis, data exploration, and extracting specific metrics for financial modeling.